In [116]:
import requests
import json
# import prettytable
import pandas as pd

####State and Area Employment, Hours and Earnings (SM) 

http://download.bls.gov/pub/time.series/sm/

from sm.txt, find series id for san francisco, hourly earnings/ weekly hours


In [117]:
# data_type_code
lookup_dataType = pd.read_csv("smdata_type.txt", delimiter="\t", index_col=False)
earnings = lookup_dataType[lookup_dataType.data_type_text.str.contains("Hourly Earnings of All Employees")].data_type_code
hours = lookup_dataType[lookup_dataType.data_type_text.str.contains("Weekly Hours of All Employees")].data_type_code

# area_code
lookup_areaCode = pd.read_csv("smarea.txt", delimiter="\t", index_col=False)
area = areaCode[areaCode.area_name.str.contains("San Francisco")].area_code

# industry_code, don't need for SF because industry is Total Private
# lookup_industry = pd.read_csv("smindustry.txt", delimiter="\t", index_col=False)

In [118]:
#find series to post requests
series = pd.read_csv("smseries.txt", delim_whitespace=True)
series_earnings = series[series.area_code.isin(area.values) & \
                         series.data_type_code.isin(earnings.values)].series_id.values
series_hours = series[series.area_code.isin(area.values) & \
                      series.data_type_code.isin(hours.values)].series_id.values

In [119]:
def get_data(seriesid_array):
    headers = {'Content-type': 'application/json', 'registrationKey': 'a4c5b701a8604c5a98f3e4a92a83c4f6'}
    data = json.dumps({"seriesid": list(seriesid_array)})
    #multiple series id use post
    p = requests.post('http://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text) 
    for s in json_data['Results']['series']:
        df = pd.DataFrame(columns=["series id","year","period","value","footnotes"])
        seriesId = s['seriesID']
        i = 0
        for item in s['data']:
            year = item['year']
            period = item['period']
            value = item['value']
            footnotes=""
            for footnote in item['footnotes']:
                if footnote:
                    footnotes = footnotes + footnote['text'] + ','
           #'if 'M01' <= period <= 'M12':'
                df.loc[i] = (seriesId,year,period,value,footnotes[0:-1])
                i+=1
    return df
#     output = open(seriesId + '.txt','w')
#     output.write (x.get_string())
#     output.close()

In [120]:
df1 = get_data(series_hours)

KeyError: 'series'